In [59]:
import numpy as np

#   Generates linear array [N x 2] = [x, y] of reflected positions
def GetPositions(nmax, dim, pos, pos0, max_dist):
    #   Generate array of positions
    a = np.array([-1, 1]).reshape((-1,1))
    x = 2*np.arange(-nmax[0], nmax[0]+1)*dim[0] + a*pos[0]
    y = 2*np.arange(-nmax[1], nmax[1]+1)*dim[1] + a*pos[1]

    #   Select those within range
    x = x[abs(x-pos0[0])<=max_dist]
    y = y[abs(y-pos0[1])<=max_dist]

    #   Generate mesh grid
    (x,y) = np.meshgrid(x, y)
    return np.vstack((x.flatten(), y.flatten()))

#   Calculates distance vectors and returns those less than distance
def GetDistances(POS, pos0, max_dist):
    #   Array if distance vectors
    delta = POS-np.array(pos0).reshape((-1,1))

    #   Squared distances
    d2 = np.sum(delta**2, axis=0)

    #   Inices where distance within range and not zero
    ind = (d2<=max_dist**2) & (d2>0)

    #   Select data subset
    return (delta[:, ind], d2[ind])

#   Find where tan(theta1) == tan(theta2)
#   Could calculate atan2(y, x), but this will convert to floating point and thus requires a tolerance setting (which could be calculated based on dim)
#   Instead I use tan(theta)=y/x ==> find (y1*x2==x1*y2) and check the quadrant of theta (i.e. relative signs of y1,y2 and x1,x2)
def TanMatrix(delta1, delta2):
    x1 = delta1[0,:]
    y1 = delta1[1,:]
    x2 = delta2[0,:].reshape((-1,1))
    y2 = delta2[1,:].reshape((-1,1))
    return ((y1*x2==x1*y2) 
            & (np.sign(y1)==np.sign(y2)) 
            & (np.sign(x1)==np.sign(x2)))

In [60]:
TEST = 1
if TEST==1:
    dim = [3,2]
    your_position = [1,1]
    trainer_position = [2,1]
    distance = 4
else:
    dim = [300, 275]
    your_position = [150, 150]
    trainer_position = [185, 100]
    distance = 500

In [61]:
#   Get maximum reflection index
nmax = np.floor(0.5*(distance + 2*np.array(your_position))/np.array(dim)).astype(int)
print(nmax)

[1 1]


In [62]:
#   Generate arrays of reflected positions for myself and trainer
POS0 = GetPositions(nmax, dim, your_position, your_position, distance)
POS1 = GetPositions(nmax, dim, trainer_position, your_position, distance)
print(POS0)
print(POS1)

[[-1  5  1 -1  5  1 -1  5  1 -1  5  1 -1  5  1]
 [-1 -1 -1  3  3  3 -3 -3 -3  1  1  1  5  5  5]]
[[-2  4  2 -2  4  2 -2  4  2 -2  4  2 -2  4  2]
 [-1 -1 -1  3  3  3 -3 -3 -3  1  1  1  5  5  5]]


In [63]:
#   Get distances of reflected points from your positions
(delta0, d20) = GetDistances(POS0, your_position, distance)
(delta1, d21) = GetDistances(POS1, your_position, distance)
print(delta0)
print(d20)
print(delta1)
print(d21)

[[-2  0 -2  0  0 -2  4  0]
 [-2 -2  2  2 -4  0  0  4]]
[ 8  4  8  4 16  4 16 16]
[[-3  3  1 -3  3  1 -3  3  1]
 [-2 -2 -2  2  2  2  0  0  0]]
[13 13  5 13 13  5  9  9  1]


In [64]:
#   Generate indices of points where !((tan(theta0)==tan(theta2) & dist0<=dist1)
ind = np.logical_not(np.any(TanMatrix(delta0, delta1) & (d20<=d21.reshape((-1,1))), axis=1))
delta1 = delta1[:, ind]
print(delta1)

[[-3  3  1 -3  3  1  3  1]
 [-2 -2 -2  2  2  2  0  0]]


In [76]:
#   Get unique directions
n = np.arange(delta1.shape[1])
ind = TanMatrix(delta1, delta1) & (n>n.reshape((-1,1)))
print(1*ind)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]]
